# Bayesian Logistic Regression (MAP + Laplace Approximation)

We model $y \in \{0,1\}$ (next-day up move) using logistic regression with a Gaussian prior.

- Likelihood: $p(y=1|x,\\beta)=\\sigma(x^T\\beta)$
- Prior: $\\beta \sim \mathcal{N}(0, \tau^2 I)$
- MAP: maximize $\\log p(\\beta|D) = \sum_i \log p(y_i|x_i,\\beta) - \frac{1}{2\\tau^2}\\|\\beta\\|^2$

Laplace approximation uses the Hessian at the MAP estimate to approximate
$p(\beta|D)$ as Gaussian.


# Chapter 10 Bayesian ML (Predictive): Bayesian Logistic Regression (MAP)

These notebooks mirror the *methods* highlighted in
`ml_finance_thoery/machine-learning-for-trading/10_bayesian_machine_learning/README.md`
and apply them to the local `dataset/cleaned/` asset universe to produce
out-of-sample predictions and a backtest using the same **vectorized** engine
used by the `notebooks/ML_Linear_Models_*` notebooks.


In [ ]:
from __future__ import annotations

import math
from pathlib import Path
import sys

import numpy as np
import pandas as pd

SEED = 42
rng = np.random.default_rng(SEED)

def find_project_root(start: Path) -> Path:
    p = start.resolve()
    for _ in range(10):
        if (p / 'src').exists() and (p / 'dataset').exists():
            return p
        p = p.parent
    raise RuntimeError(f'Could not find project root from: {start!s}')

PROJECT_ROOT = find_project_root(Path.cwd())
CLEANED_DIR = PROJECT_ROOT / 'dataset' / 'cleaned'

# Ensure `src/` is on sys.path so `backtester` is importable
src_dir = PROJECT_ROOT / 'src'
if str(src_dir) not in sys.path:
    sys.path.append(str(src_dir))


In [ ]:
from dataclasses import dataclass
from scipy.optimize import minimize
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Feature set: reuse the same SMC + OHLCV engineering pattern from ML_linear_models_05
# but keep it in-notebook for portability.

SWING_WINDOW = 5
N_CONSEC_SWINGS = 3

def compute_features(ohlcv: pd.DataFrame) -> pd.DataFrame:
    df = ohlcv.copy().sort_index()
    o = df['Open'].astype(float)
    h = df['High'].astype(float)
    l = df['Low'].astype(float)
    c = df['Close'].astype(float)
    v = df['Volume'].astype(float)
    ret_1d = c.pct_change()
    hl_range = (h - l)
    hl_range_pct = hl_range / c.replace(0.0, np.nan)
    body = (c - o)
    body_pct = body / c.replace(0.0, np.nan)
    hlc3 = (h + l + c) / 3.0
    vol_z_20 = (v - v.rolling(20).mean()) / v.rolling(20).std()

    w = int(SWING_WINDOW)
    win = 2 * w + 1
    swing_high = h.eq(h.rolling(win, center=True, min_periods=win).max())
    swing_low = l.eq(l.rolling(win, center=True, min_periods=win).min())
    last_swing_high = h.where(swing_high).ffill()
    last_swing_low = l.where(swing_low).ffill()
    prev_high = last_swing_high.shift(1)
    prev_low = last_swing_low.shift(1)
    bos_bull = (c > prev_high) & (c.shift(1) <= prev_high) & prev_high.notna()
    bos_bear = (c < prev_low) & (c.shift(1) >= prev_low) & prev_low.notna()

    # Simplified CHOCH proxy (trend not explicitly inferred here)
    choch_bull = bos_bull.astype(int)
    choch_bear = bos_bear.astype(int)

    out = pd.DataFrame({
        'open': o,
        'high': h,
        'low': l,
        'close': c,
        'volume': v,
        'ret_1d': ret_1d,
        'hl_range': hl_range,
        'hl_range_pct': hl_range_pct,
        'body': body,
        'body_pct': body_pct,
        'hlc3': hlc3,
        'vol_z_20': vol_z_20,
        'bos_bull': bos_bull.astype(int),
        'bos_bear': bos_bear.astype(int),
        'choch_bull': choch_bull,
        'choch_bear': choch_bear,
    }).sort_index()
    out['ret_1d_fwd'] = out['ret_1d'].shift(-1)
    out['y_up_fwd'] = (out['ret_1d_fwd'] > 0).astype(int)
    return out

# Load data and build a long panel (Date index, Asset_ID column)
from backtester.data import load_cleaned_assets

files = sorted([p for p in CLEANED_DIR.glob('Asset_*.csv')])
symbols = [p.stem for p in files]
assets_ohlcv = load_cleaned_assets(symbols=symbols, cleaned_dir=str(CLEANED_DIR))

frames = []
for sym, df in assets_ohlcv.items():
    feat = compute_features(df)
    feat['Asset_ID'] = sym
    frames.append(feat)

data = pd.concat(frames, axis=0).sort_index()
data = data.dropna(subset=['y_up_fwd'])

# Time split (same policy as ML_linear_models notebooks)
TRAIN_YEARS = 7
VAL_MONTHS = 18
TEST_MONTHS = 18

def align_to_trading_date(index: pd.DatetimeIndex, ts: pd.Timestamp) -> pd.Timestamp:
    pos = int(index.searchsorted(ts, side='left'))
    if pos >= len(index):
        return pd.Timestamp(index[-1])
    return pd.Timestamp(index[pos])

idx = pd.DatetimeIndex(data.index.unique()).sort_values()
end = pd.Timestamp(idx[-1])
raw_test_start = end - pd.DateOffset(months=TEST_MONTHS)
raw_val_start = raw_test_start - pd.DateOffset(months=VAL_MONTHS)
raw_train_start = raw_val_start - pd.DateOffset(years=TRAIN_YEARS)
test_start = align_to_trading_date(idx, pd.Timestamp(raw_test_start))
val_start = align_to_trading_date(idx, pd.Timestamp(raw_val_start))
train_start = align_to_trading_date(idx, pd.Timestamp(raw_train_start))
print('aligned boundaries:', train_start, val_start, test_start, end)

df_train = data.loc[(data.index >= train_start) & (data.index < val_start)].copy()
df_val = data.loc[(data.index >= val_start) & (data.index < test_start)].copy()
df_test = data.loc[(data.index >= test_start) & (data.index <= end)].copy()

feature_cols = [c for c in df_train.columns if c not in {'Asset_ID', 'ret_1d_fwd', 'y_up_fwd'} and pd.api.types.is_numeric_dtype(df_train[c])]
X_train = df_train[feature_cols].replace([np.inf, -np.inf], np.nan)
y_train = df_train['y_up_fwd'].astype(int).to_numpy()
X_test = df_test[feature_cols].replace([np.inf, -np.inf], np.nan)

# Preprocess (impute + standardize)
imp = SimpleImputer(strategy='median')
scaler = StandardScaler(with_mean=True, with_std=True)
X_train_imp = imp.fit_transform(X_train)
X_train_std = scaler.fit_transform(X_train_imp)
X_test_std = scaler.transform(imp.transform(X_test))

def sigmoid(z: np.ndarray) -> np.ndarray:
    return 1.0 / (1.0 + np.exp(-z))

# Add intercept
Xtr = np.hstack([np.ones((X_train_std.shape[0], 1)), X_train_std])
Xte = np.hstack([np.ones((X_test_std.shape[0], 1)), X_test_std])

TAU2 = 10.0**2  # prior variance

def neg_log_posterior(beta: np.ndarray) -> float:
    z = Xtr @ beta
    p = sigmoid(z)
    eps = 1e-12
    ll = np.sum(y_train * np.log(p + eps) + (1 - y_train) * np.log(1 - p + eps))
    lp = -0.5 * np.sum(beta[1:] ** 2) / TAU2  # no prior on intercept
    return float(-(ll + lp))

def grad_neg_log_posterior(beta: np.ndarray) -> np.ndarray:
    z = Xtr @ beta
    p = sigmoid(z)
    g = Xtr.T @ (p - y_train)
    g[1:] += beta[1:] / TAU2
    return g

beta0 = np.zeros(Xtr.shape[1])
res = minimize(neg_log_posterior, beta0, jac=grad_neg_log_posterior, method='BFGS')
if not res.success:
    raise RuntimeError(res.message)
beta_map = res.x
cov_laplace = res.hess_inv
print('MAP beta shape:', beta_map.shape)

# Posterior predictive via Monte Carlo from Laplace approximation
S = 200
beta_samps = rng.multivariate_normal(mean=beta_map, cov=cov_laplace, size=S)
p_samps = sigmoid(Xte @ beta_samps.T)
p_mean = p_samps.mean(axis=1)

# Convert to per-date, per-asset prediction matrix
pred_long = pd.DataFrame({
    'Date': df_test.index,
    'Asset_ID': df_test['Asset_ID'].to_numpy(),
    'p_up': p_mean,
})
pred_long['signal'] = pred_long['p_up'] - 0.5
pred_matrix = pred_long.pivot_table(index='Date', columns='Asset_ID', values='signal', aggfunc='mean').sort_index()


In [ ]:
from IPython.display import display
from bokeh.io import output_notebook, show

from backtester.data import load_cleaned_assets, align_close_prices
from backtester.engine import BacktestConfig, run_backtest
from backtester.report import compute_backtest_report
from backtester.bokeh_plots import build_interactive_portfolio_layout
from backtester.portfolio import equal_weight, optimize_mpt

output_notebook()

if 'pred_matrix' not in globals():
    raise RuntimeError('Expected `pred_matrix` (index=date, columns=Asset_ID) to exist')

# IMPORTANT: performance metrics should start when the model has signals.
# Keep the original prediction date range before reindexing to market data.
pred_range = pd.DatetimeIndex(pred_matrix.index).sort_values()
if pred_range.empty:
    raise RuntimeError('pred_matrix has empty index')
bt_start = pd.Timestamp(pred_range[0])
bt_end = pd.Timestamp(pred_range[-1])

bt_assets = sorted([str(c) for c in pred_matrix.columns.tolist()])
assets_ohlcv = load_cleaned_assets(symbols=bt_assets, cleaned_dir=str(CLEANED_DIR))
close_prices = align_close_prices(assets_ohlcv)

# Align prediction matrix to available trade dates
pred_matrix = pred_matrix.reindex(close_prices.index)
# Restrict backtest to the prediction window (avoid 2016-start metrics).
close_prices = close_prices.loc[bt_start:bt_end]
pred_matrix = pred_matrix.loc[bt_start:bt_end]
returns_matrix = close_prices.pct_change().fillna(0.0)

market_df = pd.DataFrame({
    'Open': pd.concat([df['Open'] for df in assets_ohlcv.values()], axis=1).mean(axis=1),
    'High': pd.concat([df['High'] for df in assets_ohlcv.values()], axis=1).mean(axis=1),
    'Low': pd.concat([df['Low'] for df in assets_ohlcv.values()], axis=1).mean(axis=1),
    'Close': pd.concat([df['Close'] for df in assets_ohlcv.values()], axis=1).mean(axis=1),
    'Volume': pd.concat([df['Volume'] for df in assets_ohlcv.values()], axis=1).sum(axis=1),
}).sort_index().loc[bt_start:bt_end]

REBALANCE_FREQ = 'W'
TOP_K = min(20, len(bt_assets))
LOOKBACK_DAYS = 126

def build_weights_from_predictions(pred_matrix: pd.DataFrame, *, pm_style: str) -> pd.DataFrame:
    rebal_dates = set(pd.Series(pred_matrix.index, index=pred_matrix.index).resample(REBALANCE_FREQ).last().dropna().tolist())
    w_last = pd.Series(0.0, index=bt_assets)
    rows = []
    for dt in pred_matrix.index:
        if dt in rebal_dates:
            row = pred_matrix.loc[dt].dropna().sort_values(ascending=False)
            top = row.head(TOP_K)
            candidates = [a for a, v in top.items() if np.isfinite(v) and float(v) > 0.0]
            if not candidates:
                w_last = pd.Series(0.0, index=bt_assets)
            else:
                if pm_style == '1N':
                    w_dict = equal_weight(candidates)
                elif pm_style == 'MPT':
                    w_dict = optimize_mpt(returns_matrix, candidates, dt, lookback_days=LOOKBACK_DAYS)
                else:
                    raise ValueError(f'Unknown pm_style: {pm_style!r}')
                w_last = pd.Series(0.0, index=bt_assets)
                for a, w in w_dict.items():
                    w_last[str(a)] = float(w)
        rows.append(w_last)
    return pd.DataFrame(rows, index=pred_matrix.index, columns=bt_assets).fillna(0.0)

cfg = BacktestConfig(initial_equity=1_000_000.0, transaction_cost_bps=5.0, mode='vectorized')

compare_rows = []
results = {}
for pm_style in ['1N', 'MPT']:
    w = build_weights_from_predictions(pred_matrix, pm_style=pm_style)
    res = run_backtest(close_prices, w, config=cfg)
    rpt = compute_backtest_report(result=res, close_prices=close_prices)
    results[pm_style] = (w, res, rpt)
    compare_rows.append({
        'style': pm_style,
        'Total Return [%]': float(rpt['Total Return [%]']),
        'CAGR [%]': float(rpt['CAGR [%]']),
        'Sharpe': float(rpt['Sharpe']),
        'Max Drawdown [%]': float(rpt['Max Drawdown [%]']),
    })
compare = pd.DataFrame(compare_rows).sort_values('Total Return [%]', ascending=False).reset_index(drop=True)
display(compare)

BASE_TITLE = 'Bayes Logistic Regression (MAP+Laplace)'
for pm_style in ['1N', 'MPT']:
    w, res, rpt = results[pm_style]
    title = BASE_TITLE + ' - ' + pm_style
    display(rpt.to_frame(title))
    layout = build_interactive_portfolio_layout(
        market_ohlcv=market_df,
        equity=res.equity,
        returns=res.returns,
        weights=res.weights,
        turnover=res.turnover,
        costs=res.costs,
        close_prices=close_prices,
        title=title,
    )
    show(layout)
